In [1]:
import pickle
import bs4
import gzip
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import pprint

from bs4 import BeautifulSoup
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from pprint import pprint
from operator import itemgetter

from bs4 import BeautifulSoup

import os

In [2]:
#
# Get the content of all members in a tarfile folder
# tar_file = the tar file path
# fold_name = the prefix of the path - eg. 'tom-forums/2014-12-02'
#

tar_file = "/home/hades/exjobb/dumps/dnm/tom-forums.tar.xz"
fold_name = 'tom-forums/2014-12-02'

In [54]:
#
# Get all files in a specified folder (prefix specified) 
# Arg: filename(str), foldername(str)  
# Return: members
#

def get_tar_folder(tar_file, folder_name):
    with tarfile.open(tar_file, "r:xz") as tar:
        folder = []
        for tarinfo in tar.getmembers():
            if tarinfo.name.startswith(fold_name):
                folder.append(tarinfo)
    return(folder)

folder = get_tar_folder(tar_file, fold_name)
print("Files:", len(list(set(folder))))

Files: 1039


In [59]:
#
# Get contentent of a list of tarinfo-files
# Arg: members  
# Return: content (list of str)
#

def get_content(members):
    cont = []
    with tarfile.open(tar_file, "r:xz") as tar:
        for ti in tqdm(members):
            with tar.extractfile(ti) as f_like:
                raw = f_like.read()
                cont.append(raw)
    return(cont)

content = get_content(folder)

print("Files:", len(list(set(content))))

100%|██████████| 1039/1039 [00:04<00:00, 252.91it/s]

Files: 1032


In [45]:
#
# Get categories from silkroad
#

root = "/home/hades/exjobb/dumps/dnm/silkroad/2014-06-24/items/"
f_l = []
for path, subdirs, files in os.walk(root):
    for name in files:
        f_l.append(os.path.join(path, name))

print("files:",len(list(set(f_l))))

files: 3169


In [4]:
def get_c(fn):
    with open(fn, "rb") as f:
        raw = f.read()
    return(raw)


In [6]:
def get_itemlist(root_cat,fn):
    with open(root_cat+fn, "rb") as f:
        raw = f.read()

    soup = BeautifulSoup(raw, 'lxml')

    topic_raw = [link.get('href') for link in soup.find_all('a')]

    if topic_raw == None:
        return([])

    topic = list(set([x for x in topic_raw if None != x and '/item' == x[:5] and not 'shipping' == x[-8:]]))

    return(topic)

# get_itemlist("drugs-cannabis-shake/items")

In [7]:
def get_category_lists(root_cat):
    categories = []
    for path, subdirs, files in os.walk(root_cat):
        for name in files:
            tmp = os.path.join(path, name)
            tmp = tmp.replace(root_cat, "")
            categories.append(tmp)
    return(categories)

# get_category_lists()

In [86]:
#
# Collects the 
#

root_cat = "/home/hades/exjobb/dumps/dnm/silkroad/2014-06-24/categories/"

cat_cont = {}
for c in tqdm(get_category_lists(root_cat)):
    c_ = c.split("/")[:-1][0]
    c_ = c_.split("-")[0]
    if c_ == 'drug':
        c_+='s'
    try:
        cat_cont[c_] += get_itemlist(root_cat, c)
    except:
        cat_cont[c_] = get_itemlist(root_cat, c)

100%|██████████| 312/312 [00:04<00:00, 69.02it/s]


In [90]:
#
# Removes script and html-tags - returns plain text
#
def clean_html(str):
    soup = BeautifulSoup(str,"lxml")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract() # rip it out 
    text = soup.get_text(strip=False)
    text = text.replace(u'\xa0',u'')
#     text = rem_numb(text)
    return text

In [91]:
# print(len(cat_cont.get("drugs-ecstasy-mdai")))

# pprint(cat_cont.get("drugs-ecstasy-mdai"))
goo = 0
err = 0

conts = {}

for x in tqdm(cat_cont.keys()):
    tmp = []
    for k in cat_cont.get(x):
        try:
            with open(root_cat[:-12]+k,"rb") as f:
                raw = f.read()
                clean = clean_html(raw)
                tmp.append(clean)
            goo+=1
        except:
            err+=1
    conts[x] = tmp
    
print(err, goo)

100%|██████████| 22/22 [00:26<00:00,  1.61it/s]

3347 2137


In [96]:
# for x in cat_cont.keys():
#     print(len(cat_cont.get(x)),x)
    

for x in conts.keys():
    print(len(conts.get(x)),x)
    
print(conts.get("drugs")[23])

# conts.keys()

26 money
3 collectibles
24 packaging
44 forgeries
9 art
23 electronics
19 lotteries
26 apparel
25 jewelry
10 hardware
24 lab
24 services
7 medical
1716 drugs
22 computer
25 erotica
2 biotic
25 books
24 writing
32 custom
2 herbs
25 digital






      Silk Road
    









Silk Road
anonymous market





          messages
          0



          orders
          0



          account
          ฿0.000







0




          Hi, 
          gwern
        settings - logout





            Search
          














Heroin
 144

    Black tar
 10
    Brown
 36
    White
 38
Alcohol
 406
Apparel
 477
Art
 10
Biotic materials
 2
Books
 552
Collectibles
 3
Computer equipment
 23
Custom Orders
 271
Digital goods
 764
Drug paraphernalia
 220
Drugs
 13572
Electronics
 33
Erotica
 95
Forgeries
 105
Hardware
 10
Herbs & Supplements
 2
Jewelry
 38
Lab Supplies
 28
Lotteries & games
 20
Medical
 8
Money
 336
Packaging
 35
Services
 195
Writing
 47





250mg sample Afghan Heroin

    ฿0.049

In [104]:
# pickle.dump(conts, open("silkroad_dict.p", "wb"))

In [113]:
res = []

for x in conts.keys():
    for text in conts.get(x):
        res_ = {}
        res_['description'] = text
        res_['category'] = x
        res.append(res_)

In [114]:
# pickle.dump(res, open("dataset_silkroad.p", "wb"))